In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

t1 = PromptTemplate.from_template("What is the capital of {country}")

t1.format(country="Korea")

t2 = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
)

t2.format(country="Korea")

# t1 === t2


'What is the capital of Korea'

In [12]:
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
        {
            "question": "What do you know about France?",
            "answer": """
            Here is what I know:
            Capital: Parisn
            Language: Frenchn
            Food: Wine and Cheesen
            Currency: Euron
            """,
        },
        {
            "question": "What do you know about Italy?",
            "answer": """
           I know this:
            Capital: Romen
            Language: Italiann
            Food: Pizza and Pastan
            Currency: Euron
            """,
        },
        {
            "question": "What do you know about Greece?",
            "answer": """
            I know this:
            Capital: Athensn
            Language: Greekn
            Food: Souvlaki and Feta Cheesen
            Currency: Euron
            """,
        },
    ]


class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template);
# example_prompt = PromptTemplate.from_template("Human: {question}nAI:{answer}");

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=160,
)

random_example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=random_example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")




'\n    Human: What do you know about Greece?\n    AI: \n            I know this:\n            Capital: Athensn\n            Language: Greekn\n            Food: Souvlaki and Feta Cheesen\n            Currency: Euron\n            \n\n\nHuman: What do you know about Germany?'

In [5]:
chain = prompt | chat

chain.invoke({"country": "Korea"})

AI: 
            Here is what I know:
            Capital: Seoul
            Language: Korean
            Food: Kimchi and Bulgogi
            Currency: South Korean Won

AIMessageChunk(content='AI: \n            Here is what I know:\n            Capital: Seoul\n            Language: Korean\n            Food: Kimchi and Bulgogi\n            Currency: South Korean Won')

In [3]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
        {
            "country": "France",
            "answer": """
            Here is what I know:
            Capital: Parisn
            Language: Frenchn
            Food: Wine and Cheesen
            Currency: Euron
            """,
        },
        {
            "country": "Italy",
            "answer": """
           I know this:
            Capital: Romen
            Language: Italiann
            Food: Pizza and Pastan
            Currency: Euron
            """,
        },
        {
            "country": "Greece",
            "answer": """
            I know this:
            Capital: Athensn
            Language: Greekn
            Food: Souvlaki and Feta Cheesen
            Currency: Euron
            """,
        },
    ]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({"country": "Germany"})


            I know this:
            Capital: Berlinn
            Language: Germann
            Food: Bratwurst and Sauerkrautn
            Currency: Euro

AIMessageChunk(content='\n            I know this:\n            Capital: Berlinn\n            Language: Germann\n            Food: Bratwurst and Sauerkrautn\n            Currency: Euro')